# Prepare Datasets
To de-normalize SalesOrderHeader and SalesOrderDetail datasets<br>
- **Sales Order Header** combined with;
 - SalesTerritory
 - Customer
 - Person
 - **Target:** STG_SalesOrderHeader.csv
- **Sales Order Detail** combined with;
 - Product
 - ProdcutSubcategory
 - ProductCategroy
 - SpecialOffer
 - **Target:** STG_SalesOrderDetail.csv

In [69]:
import pandas as pd

# SalesOrderHeader processing

### SalesTerritory

In [70]:
terr = pd.read_csv('datasets/SalesTerritory.csv', usecols=[0,1,2], index_col=0, 
                   names=['TerritoryID','Territory','Country'], header=0)
terr.head()

,Territory,Country
TerritoryID,,
1,Northwest,US
2,Northeast,US
3,Central,US
4,Southwest,US
5,Southeast,US


### Customer

In [71]:
cust = pd.read_csv('datasets/Customer.csv', index_col=0, usecols=[0,1,4])
# cust['PersonID'].value_counts(dropna=False).head()
# cust.loc['29825',:]
cust.head()

,PersonID,AccountNumber
CustomerID,,
1,NaN,AW00000001
2,NaN,AW00000002
3,NaN,AW00000003
4,NaN,AW00000004
5,NaN,AW00000005


### Person

In [72]:
persons = pd.read_csv('datasets/Person.csv', usecols=[0,1,4,6], index_col=0)
persons.head()

,PersonType,FirstName,LastName
BusinessEntityID,,,
1,EM,Ken,Sánchez
2,EM,Terri,Duffy
3,EM,Roberto,Tamburello
4,EM,Rob,Walters
5,EM,Gail,Erickson


In [73]:
print('FirstName null count: {}'.format(persons.FirstName.isnull().sum()))
print('LastName null count: {}'.format(persons.FirstName.isnull().sum()))

FirstName null count: 0
LastName null count: 0


In [74]:
persons['PersonName'] = persons['FirstName'] + ' ' + persons['LastName']
persons.drop(['FirstName','LastName'], axis='columns', inplace=True)
persons.head()

,PersonType,PersonName
BusinessEntityID,,
1,EM,Ken Sánchez
2,EM,Terri Duffy
3,EM,Roberto Tamburello
4,EM,Rob Walters
5,EM,Gail Erickson


## SalesOrderHeader

In [75]:
soh_cols = ['SalesOrderID','OrderDate', 'OnlineOrderFlag',
            'CustomerID','SalesPersonID','TerritoryID',
            'SubTotal','TaxAmt','Freight','TotalDue']

soh = pd.read_csv('datasets/SalesOrderHeader.csv', usecols=soh_cols, parse_dates=['OrderDate'], index_col=0)
soh.head()

,OrderDate,OnlineOrderFlag,CustomerID,SalesPersonID,TerritoryID,SubTotal,TaxAmt,Freight,TotalDue
SalesOrderID,,,,,,,,,
43659,2011-05-31,False,29825,279.0,5,20565.6206,1971.5149,616.0984,23153.2339
43660,2011-05-31,False,29672,279.0,5,1294.2529,124.2483,38.8276,1457.3288
43661,2011-05-31,False,29734,282.0,6,32726.4786,3153.7696,985.5530,36865.8012
43662,2011-05-31,False,29994,282.0,6,28832.5289,2775.1646,867.2389,32474.9324
43663,2011-05-31,False,29565,276.0,4,419.4589,40.2681,12.5838,472.3108


### Joining SalesOrderHeader and supplemental info

In [76]:
stg_soh = soh.join(cust, on='CustomerID', how='left') \
        .join(persons, on='PersonID', how='left') \
        .join(persons['PersonName'], on='SalesPersonID', how='left', rsuffix='SP') \
        .join(terr, on='TerritoryID', how='left')

# rename some columns
stg_soh.rename({'PersonType': 'CustType', 'PersonName': 'CustName', 'PersonNameSP':'SalesPersonName'}, axis=1, inplace=True)
# drop some columns
stg_soh.drop(['CustomerID','SalesPersonID','TerritoryID','PersonID'], axis=1, inplace=True)
stg_soh.head(20)

,OrderDate,OnlineOrderFlag,SubTotal,TaxAmt,Freight,TotalDue,AccountNumber,CustType,CustName,SalesPersonName,Territory,Country
SalesOrderID,,,,,,,,,,,,
43659,2011-05-31,False,20565.6206,1971.5149,616.0984,23153.2339,AW00029825,SC,James Hendergart,Tsvi Reiter,Southeast,US
43660,2011-05-31,False,1294.2529,124.2483,38.8276,1457.3288,AW00029672,SC,Takiko Collins,Tsvi Reiter,Southeast,US
43661,2011-05-31,False,32726.4786,3153.7696,985.5530,36865.8012,AW00029734,SC,Jauna Elson,José Saraiva,Canada,CA
43662,2011-05-31,False,28832.5289,2775.1646,867.2389,32474.9324,AW00029994,SC,Robin McGuigan,José Saraiva,Canada,CA
43663,2011-05-31,False,419.4589,40.2681,12.5838,472.3108,AW00029565,SC,Jimmy Bischoff,Linda Mitchell,Southwest,US
43664,2011-05-31,False,24432.6088,2344.9921,732.8100,27510.4109,AW00029898,SC,Sandeep Katyal,Pamela Ansman-Wolfe,Northwest,US
43665,2011-05-31,False,14352.7713,1375.9427,429.9821,16158.6961,AW00029580,SC,Richard Bready,David Campbell,Northwest,US
43666,2011-05-31,False,5056.4896,486.3747,151.9921,5694.8564,AW00030052,SC,Abraham Swearengin,Linda Mitchell,Southwest,US
43667,2011-05-31,False,6107.0820,586.1203,183.1626,6876.3649,AW00029974,SC,Scott MacDonald,Jillian Carson,Central,US


In [77]:
# save!
stg_soh.to_csv('staging/STG_SalesOrderHeader.csv')

# SalesOrderDetail processing

### Product

In [78]:
cols = [0, 1, 8, 9, 18, 19]
products = pd.read_csv('datasets/Product.csv', usecols=cols, index_col = 0)
products.rename({'Name':'Product'}, axis='columns', inplace=True)
# _ = [ print('{}: {}'.format(k,v)) for k,v in enumerate(products.columns)]
products.head()

,Product,StandardCost,ListPrice,ProductSubcategoryID,ProductModelID
ProductID,,,,,
1,Adjustable Race,0.0,0.0,NaN,NaN
2,Bearing Ball,0.0,0.0,NaN,NaN
3,BB Ball Bearing,0.0,0.0,NaN,NaN
4,Headset Ball Bearings,0.0,0.0,NaN,NaN
316,Blade,0.0,0.0,NaN,NaN


### ProductSubCategory

In [79]:
psubcats = pd.read_csv('datasets/ProductSubCategory.csv', usecols=[0,1,2], index_col=0)
psubcats.rename({'Name':'ProductSubcategory'}, axis='columns', inplace=True)
psubcats.head()

,ProductCategoryID,ProductSubcategory
ProductSubcategoryID,,
1,1,Mountain Bikes
2,1,Road Bikes
3,1,Touring Bikes
4,2,Handlebars
5,2,Bottom Brackets


### ProductCategory

In [80]:
pcats = pd.read_csv('datasets/ProductCategory.csv', usecols=[0,1], index_col=0)
pcats.columns = ['ProductCategory']
pcats.head()

,ProductCategory
ProductCategoryID,
1,Bikes
2,Components
3,Clothing
4,Accessories


In [81]:
pmod = pd.read_csv('datasets/ProductModel.csv', index_col=0)
pmod.columns = ['ProductModel']
pmod.head()

,ProductModel
ProductModelID,
122,All-Purpose Bike Stand
119,Bike Wash
115,Cable Lock
98,Chain
1,Classic Vest


### Merge Product master

In [82]:
stg_products = products.join(psubcats, on='ProductSubcategoryID', how='left') \
    .join(pcats, on='ProductCategoryID', how='left') \
    .join(pmod, on='ProductModelID', how='left')

stg_products.drop(['ProductSubcategoryID','ProductCategoryID','ProductModelID'], axis=1, inplace=True)
stg_products #.head(100)

,Product,StandardCost,ListPrice,ProductSubcategory,ProductCategory,ProductModel
ProductID,,,,,,
1,Adjustable Race,0.0000,0.00,NaN,NaN,NaN
2,Bearing Ball,0.0000,0.00,NaN,NaN,NaN
3,BB Ball Bearing,0.0000,0.00,NaN,NaN,NaN
4,Headset Ball Bearings,0.0000,0.00,NaN,NaN,NaN
316,Blade,0.0000,0.00,NaN,NaN,NaN
317,LL Crankarm,0.0000,0.00,NaN,NaN,NaN
318,ML Crankarm,0.0000,0.00,NaN,NaN,NaN
319,HL Crankarm,0.0000,0.00,NaN,NaN,NaN
320,Chainring Bolts,0.0000,0.00,NaN,NaN,NaN


In [83]:
# just in case
stg_products.to_csv('staging/STG_ProductMaster.csv')

### SpecialOffer

In [84]:
spoffer = pd.read_csv('datasets/SpecialOffer.csv', index_col=0, usecols=[0,1,3], 
                      names=['SpecialOfferID','SpecialOffer','SpecialOfferType'],
                     header=0)
spoffer.head()

,SpecialOffer,SpecialOfferType
SpecialOfferID,,
1,No Discount,No Discount
2,Volume Discount 11 to 14,Volume Discount
3,Volume Discount 15 to 24,Volume Discount
4,Volume Discount 25 to 40,Volume Discount
5,Volume Discount 41 to 60,Volume Discount


## SalesOrderDetail

In [85]:
sod = pd.read_csv('datasets/SalesOrderDetail.csv', usecols=[0,1,3,4,5,6,7,8], index_col=1)
sod.head()

,SalesOrderID,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal
SalesOrderDetailID,,,,,,,
1,43659,1,776,1,2024.994,0.0,2024.994
2,43659,3,777,1,2024.994,0.0,6074.982
3,43659,1,778,1,2024.994,0.0,2024.994
4,43659,1,771,1,2039.994,0.0,2039.994
5,43659,1,772,1,2039.994,0.0,2039.994


In [86]:
stg_sod = sod.join(stg_products, on='ProductID', how='inner') \
    .join(spoffer, on='SpecialOfferID', how='left')

stg_sod.drop(['ProductID','SpecialOfferID'], axis=1, inplace=True)
stg_sod.sort_index(inplace=True)

stg_sod.head()

,SalesOrderID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,Product,StandardCost,ListPrice,ProductSubcategory,ProductCategory,ProductModel,SpecialOffer,SpecialOfferType
SalesOrderDetailID,,,,,,,,,,,,,
1,43659,1,2024.994,0.0,2024.994,"Mountain-100 Black, 42",1898.0944,3374.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
2,43659,3,2024.994,0.0,6074.982,"Mountain-100 Black, 44",1898.0944,3374.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
3,43659,1,2024.994,0.0,2024.994,"Mountain-100 Black, 48",1898.0944,3374.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
4,43659,1,2039.994,0.0,2039.994,"Mountain-100 Silver, 38",1912.1544,3399.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
5,43659,1,2039.994,0.0,2039.994,"Mountain-100 Silver, 42",1912.1544,3399.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount


In [87]:
stg_sod.to_csv('staging/STG_SalesOrderDetail.csv')

## Sales Order Header and Details combined

In [88]:
stg_final = pd.merge(stg_soh.reset_index(), stg_sod.reset_index(), on='SalesOrderID', how='inner')
stg_final.head()

,SalesOrderID,OrderDate,OnlineOrderFlag,SubTotal,TaxAmt,Freight,TotalDue,AccountNumber,CustType,CustName,...,UnitPriceDiscount,LineTotal,Product,StandardCost,ListPrice,ProductSubcategory,ProductCategory,ProductModel,SpecialOffer,SpecialOfferType
0,43659,2011-05-31,False,20565.6206,1971.5149,616.0984,23153.2339,AW00029825,SC,James Hendergart,...,0.0,2024.994,"Mountain-100 Black, 42",1898.0944,3374.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
1,43659,2011-05-31,False,20565.6206,1971.5149,616.0984,23153.2339,AW00029825,SC,James Hendergart,...,0.0,6074.982,"Mountain-100 Black, 44",1898.0944,3374.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
2,43659,2011-05-31,False,20565.6206,1971.5149,616.0984,23153.2339,AW00029825,SC,James Hendergart,...,0.0,2024.994,"Mountain-100 Black, 48",1898.0944,3374.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
3,43659,2011-05-31,False,20565.6206,1971.5149,616.0984,23153.2339,AW00029825,SC,James Hendergart,...,0.0,2039.994,"Mountain-100 Silver, 38",1912.1544,3399.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount
4,43659,2011-05-31,False,20565.6206,1971.5149,616.0984,23153.2339,AW00029825,SC,James Hendergart,...,0.0,2039.994,"Mountain-100 Silver, 42",1912.1544,3399.99,Mountain Bikes,Bikes,Mountain-100,No Discount,No Discount


In [92]:
stg_final.to_csv('staging/STG_Sales.csv', index=False)

In [102]:
'{} MB'.format(stg_final.memory_usage().sum() / 1024**2)

'18.627202033996582 MB'